In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import random
import json
from scholarly import scholarly
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [3]:
driver = webdriver.Chrome(r"C:\Users\COM01\Downloads\chromedriver_win32(1)\chromedriver.exe", chrome_options=chrome_options)
driver.get('https://scholar.google.com/')
# log-in by self

C:\Users\COM01\AppData\Local\Temp\ipykernel_19076\4208443415.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\COM01\Downloads\chromedriver_win32(1)\chromedriver.exe", chrome_options=chrome_options)
C:\Users\COM01\AppData\Local\Temp\ipykernel_19076\4208443415.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"C:\Users\COM01\Downloads\chromedriver_win32(1)\chromedriver.exe", chrome_options=chrome_options)


In [4]:
# read csv file 
# list scopus id into list_id
df = pd.read_csv(r"C:\Users\COM01\Downloads\5065-6330.csv",encoding='latin1')
list_id = ["Saowanit Chunlawong"]

In [5]:
scholar = []
for id in list_id:
    driver.get(f'https://scholar.google.com/scholar?hl=en&as_sdt=0%2C5&q')
    name = list_id
    search_input = driver.find_element("name", "q")
    search_input.send_keys(name)
    search_input.send_keys(Keys.RETURN)
    time.sleep(3)
    
    next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gs_nm > button.gs_btnPR.gs_in_ib.gs_btn_lrge.gs_btn_half.gs_btn_lsu')))
    while next_button:
    # find all the articles on the current page
        articles = driver.find_elements(By.XPATH, '//*[@id="gs_bdy"]')
        
        for article in articles:
            try:
                author_name = article.find_element(By.XPATH, '//*[@id="gs_res_ccl_mid"]/div[1]/h3/a/b').text
            except  NoSuchElementException:
                pass
 
            research_list = []
            researchs = article.find_elements(By.XPATH, '//*[@id="y5aWsq4_S0wJ"]')
            for i in range(len(researchs)):
                research_list.append(researchs[i].text)
    
            date_list = []
            research_date = article.find_elements(By.XPATH, '//*[@id="gs_res_ccl_mid"]/div[2]/div[2]/div[1]')
            for i in range(len(research_date)):
                date_list.append(research_date[i].text)

                abs_list = []
                abs = article.find_elements(By.XPATH, '//*[@id="gs_res_ccl_mid"]/div[2]/div[2]/div[2]')
                for i in range(len(abs)):
                    abs_list.append(abs[i].text)
    
            scopus = dict()
            scopus['author_name'] = author_name
            scopus['documents'] = []
            for i in range(len(research_list)):
                temp_dict = dict()
                temp_dict['title'] = research_list[i]
                temp_dict['date'] = date_list[i]
                temp_dict['abstraction'] = abs_list[i]
                scopus['documents'].append(temp_dict)

            scholar.append(scopus)

        next_button.click()

        try:
            # wait for the "next" button element to become available again
            next_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#gs_nm > button.gs_btnPR.gs_in_ib.gs_btn_lrge.gs_btn_half.gs_btn_lsu')))
        except:
            break



In [ ]:
scholar

[]

In [ ]:
df1 = pd.DataFrame(scholar, columns = ['author_id', 'author_name', 'documents_number',
                                           'citiations' , 'h-index' , 'co-authors' , 'documents'])

In [ ]:
df1

In [ ]:
df1['documents'][0]

In [ ]:
type(df1['documents'][0])

In [ ]:
df1['documents'][0][1]

In [ ]:
df1=df1.explode('documents')
df1

In [ ]:
df1['main_title']=df1['documents'].apply(lambda v:v)
df1

In [ ]:
df1['main_abstract']=df1['main_title'].apply(lambda v:v)
df1

In [ ]:
df1['main_title'].apply(lambda v: v['title'])

In [ ]:
df1['Title']=df1['main_title'].apply(lambda v:v['title'])
df1

In [ ]:
df1['main_abstract'].apply(lambda v: v['abstraction'])

In [ ]:
df1['Abstraction']=df1['main_abstract'].apply(lambda v:v['abstraction'])
df1

In [ ]:
df1.drop(['documents' , 'main_title' , 'main_abstract' , 'documents_number'], axis=1)

In [ ]:
df1.to_xlsx('resercher_all.xlsx',index=False)